In [1]:
import pandas as pd
import plotly.graph_objects as go
from signals import *

signal_file = "./telegram/2020-21-01_TelegramDump.json"

In [2]:
def analyse_signal(df, signal):
    """
    Process signal using SL and TP.
    TODO: accound for spread and fees
    TODO: Warn if slippage is high
    """
    real_entry = signal.real_entry_price
    
    for dt, ohlc in df.loc[signal.datetime_str:].iterrows():
        if signal.type == "BUY":
            if ohlc.High > signal.take_profit:
                return signal.take_profit - real_entry

            if ohlc.Low < signal.stop_loss:
                return signal.stop_loss - real_entry
            
        if signal.type == "SELL":
            if ohlc.Low < signal.take_profit:
                return real_entry - signal.take_profit

            if ohlc.High > signal.stop_loss:
                return real_entry - signal.stop_loss
        
    return 0.0
        

In [3]:
beta_tp1 = Signal(ticker="AUDCAD", datetime_str="2020-01-14 13:19", 
                  entry_price=0.9020, stop_loss=0.9075, take_profit=0.9000)
df = load_ticker(beta_tp1.ticker)
print("TP1", analyse_signal(df, beta_tp1) * pip_factor(beta_tp1.ticker))

TP1 20.000000000000018


In [4]:
# Single run Analysis report
signals = load_beta_trades(signal_file)
total_profit = 0.0
wins = 0
losses = 0
for signal in signals[-100:]:
    df = load_ticker(signal.ticker, 60)
    #signal.set_stop_loss_pips(24)
    signal.set_take_profit_pips(51)
    
    profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
    if profit > 0:
        wins += 1
    else:
        losses += 1
    total_profit += profit
    print(f"{signal}:  {profit:.2f}")
print("Default:", total_profit, wins, losses)

Slippage to high for: GBPCAD: 2019-04-30T15:35:43: Entry: 1.75180, SL: 1.77550, TP: 0.00000
Slippage to high for: GBPNZD: 2019-04-30T15:35:43: Entry: 1.95280, SL: 1.97100, TP: 0.00000
Slippage to high for: GBPAUD: 2019-05-01T21:20:21: Entry: 1.85990, SL: 1.87400, TP: 0.00000
Slippage to high for: GBPCAD: 2019-05-01T21:20:48: Entry: 1.75390, SL: 1.77100, TP: 0.00000
Slippage to high for: EURNZD: 2019-05-07T13:51:44: Entry: 1.69610, SL: 1.70900, TP: 0.00000
Slippage to high for: EURGBP: 2019-05-14T14:39:23: Entry: 0.86790, SL: 1.87450, TP: 0.00000
Slippage to high for: EURJPY: 2019-05-15T15:15:11: Entry: 122.91000, SL: 1.23500, TP: 0.00000
Slippage to high for: EURAUD: 2019-05-21T04:13:52: Entry: 1.61820, SL: 1.62750, TP: 0.00000
Slippage to high for: GBPCHF: 2019-05-21T16:12:09: Entry: 1.28870, SL: 1.27000, TP: 0.00000
Slippage to high for: XAUUSD: 2019-06-05T13:54:21: Entry: 1341.40000, SL: 1349.50000, TP: 0.00000
Slippage to high for: USDCAD: 2019-06-19T13:32:35: Entry: 1.33430, SL: 1

NZDUSD: 2019-10-24T21:58:25: Entry: 0.63790, SL: 0.64450, TP: 0.63290:  -66.00
EURJPY: 2019-10-29T22:28:44: Entry: 120.99000, SL: 121.30000, TP: 120.49000:  -31.00
NZDJPY: 2019-10-29T22:40:09: Entry: 69.15000, SL: 69.70000, TP: 68.69000:  -55.00
NZDCHF: 2019-10-29T22:43:29: Entry: 0.63130, SL: 0.63900, TP: 0.62690:  -77.00
EURCHF: 2019-10-30T10:23:34: Entry: 1.10240, SL: 1.10870, TP: 1.09740:  50.00
NZDCHF: 2019-10-30T10:24:50: Entry: 0.63020, SL: 0.63700, TP: 0.62540:  -68.00
USDCAD: 2019-10-30T10:26:10: Entry: 1.30770, SL: 1.30180, TP: 1.31290:  52.00
AUDCHF: 2019-11-04T13:36:17: Entry: 0.68180, SL: 0.68800, TP: 0.67690:  -62.00
AUDCAD: 2019-11-04T13:37:14: Entry: 0.90780, SL: 0.91500, TP: 0.90340:  44.00
NZDUSD: 2019-11-04T13:37:56: Entry: 0.64240, SL: 0.63800, TP: 0.64780:  -44.00
AUDJPY: 2019-11-04T13:39:08: Entry: 74.85000, SL: 75.50000, TP: 74.36000:  -65.00
USDCAD: 2019-11-04T16:40:14: Entry: 1.31450, SL: 1.32200, TP: 1.30980:  -75.00
GBPJPY: 2019-11-05T08:01:19: Entry: 140.280

In [7]:
# Take Profit analysis
signals = load_beta_trades(signal_file)

stop_loss = 45
take_profits = range(10, 100, 1)
profits = []

for take_profit in take_profits:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals[-100:]:
        signal.set_stop_loss_pips(stop_loss)
        signal.set_take_profit_pips(take_profit)
        df = load_ticker(signal.ticker, 60)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(take_profit, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(take_profits), y=profits))
fig.update_layout(
    title=f"BETA: Take Profit Optimisation (Stop Loss = {stop_loss} pips)",
    title_x=0.5,
    xaxis_title="Take Profit (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()
    

10 253.4999999999528 89 11
11 286.50000000003337 88 12
12 317.50000000004 87 13
13 404.500000000005 87 13
14 491.50000000001177 87 13
15 518.5000000000259 86 14
16 604.5000000000177 86 14
17 628.5000000000241 85 15
18 713.5000000000368 85 15
19 798.5000000000048 85 15
20 883.5000000000089 85 15
21 968.4999999999837 85 15
22 1053.4999999999895 85 15
23 1070.4999999999939 84 16
24 1154.4999999999707 84 16
25 1238.4999999999748 84 16
26 1251.5000000000055 83 17
27 1262.4999999999845 82 18
28 1271.4999999999875 81 19
29 1179.4999999999939 78 22
30 1107.4999999999732 76 24
31 1183.5000000000384 76 24
32 1259.5000000000107 76 24
33 1257.500000000023 75 25
34 1332.5000000000277 75 25
35 1327.4999999999977 74 26
36 1239.5000000000314 72 28
37 1311.5000000000373 72 28
38 1383.5000000000077 72 28
39 1287.500000000014 70 30
40 1272.5000000000248 69 31
41 1341.4999999999968 69 31
42 1323.5000000000036 68 32
43 1303.500000000013 67 33
44 1192.4999999999889 65 35
45 1257.499999999992 65 35
46 1322.4

In [6]:
# Stop Loss analysis
signals = load_beta_trades(signal_file)

take_profit = 25
stops = range(10, 100, 1)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals[-100:]:
        signal.set_stop_loss_pips(stop)
        signal.set_take_profit_pips(take_profit)
        df = load_ticker(signal.ticker, 60)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title=f"BETA: Stop Loss Optimisation (Take Profit = {take_profit} pips)",
    title_x=0.5,
    xaxis_title="Stop Loss Level (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()
    

10 258.5000000000216 40 60
11 306.49999999996993 43 57
12 434.4999999999686 48 52
13 458.4999999999858 50 50
14 486.49999999998386 52 48
15 518.4999999999748 54 46
16 513.4999999999797 55 45
17 552.4999999999709 57 43
18 509.4999999999686 57 43
19 554.4999999999793 59 41
20 513.4999999999797 59 41
21 564.4999999999916 61 39
22 525.4999999999826 61 39
23 582.4999999999839 63 37
24 594.4999999999892 64 36
25 608.4999999999893 65 35
26 675.4999999999741 67 33
27 642.4999999999836 67 33
28 609.4999999999807 67 33
29 576.4999999999775 67 33
30 543.4999999999873 67 33
31 510.4999999999709 67 33
32 591.4999999999798 69 31
33 734.4999999999768 72 28
34 765.4999999999702 73 27
35 798.4999999999834 74 26
36 833.4999999999695 75 25
37 932.4999999999675 77 23
38 909.4999999999761 77 23
39 950.4999999999725 78 22
40 928.4999999999693 78 22
41 1038.4999999999761 80 20
42 1018.4999999999741 80 20
43 1134.4999999999698 82 18
44 1185.499999999978 83 17
45 1238.4999999999748 84 16
46 1222.4999999999757 